<a href="https://colab.research.google.com/github/chaggerty93/Gaza-Solidarity-Encampments-Map/blob/main/Gaza_Solidarity_Encampment_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import folium
from folium import IFrame
import geopandas
import gspread
import gspread_dataframe as gdf
from oauth2client.client import GoogleCredentials
from google.colab import drive, auth
import pandas as pd
import base64
import re
import os
import imageio
import cv2
import glob
from google.auth import default


In [59]:
drive.mount('/content/drive', force_remount=True)

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Encampments').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_records()

# Convert to a DataFrame and render.
encampment_data = pd.DataFrame(rows)
encampment_data = encampment_data.convert_dtypes()
encampment_data.dtypes

encampment_data['Latitude'] = pd.to_numeric(encampment_data['Latitude'], errors='coerce')
encampment_data['Longitude'] = pd.to_numeric(encampment_data['Longitude'], errors='coerce')
encampment_data['Encampment Start Date'] = pd.to_datetime(encampment_data['Encampment Start Date'], errors='coerce')
encampment_data['Date of raid/arrests'] = pd.to_datetime(encampment_data['Date of raid/arrests'], errors='coerce')
encampment_data['Location'] = encampment_data['City'].str.strip() + ", " + encampment_data['State'].str.strip()
encampment_data


Mounted at /content/drive


,University Name,City,State,Country,Latitude,Longitude,Photo Location,Category,Status,Encampment Start Date,Police/State Violence,Image Source,Date of raid/arrests,Number of Arrests,Police Violence,,Source,Location
0,Columbia University & Barnard College,New York,New York,U.S.,40.807384,-73.963036,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,2024-04-17,Mass arrests,PYM NYC,2024-04-18,114+,,,https://www.instagram.com/reel/C53zTS0uLxg/?ut...,"New York, New York"
1,The New School,New York,New York,U.S.,40.734677,-73.991509,https://drive.google.com/drive/folders/12-V7pk...,Encampment,Encampment Ongoing,2024-04-21,,TNS SJP,NaT,NA,,,https://www.instagram.com/reel/C6JjZykLzGB/?ut...,"New York, New York"
2,New York University (NYU),New York,New York,U.S.,40.729675,-73.996925,https://drive.google.com/drive/folders/12-V7pk...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",2024-04-22,Mass arrests,National SJP,2024-04-22,115+,@AshAgony,,https://www.instagram.com/p/C6DrwzRuxhV/?utm_s...,"New York, New York"
3,Yale University,New Haven,Connecticut,U.S.,41.316307,-72.922585,https://drive.google.com/drive/folders/1x071zz...,Encampment,Encampment Dispersed,2024-04-20,Mass arrests,Yalies4palestine,2024-04-22,47+,,,https://www.instagram.com/p/C6EF_mnL9k0/,"New Haven, Connecticut"
4,University of Minnesotta,Twin Cities,Minnesotta,U.S.,44.974000,-93.227700,https://drive.google.com/drive/folders/1U_tKJC...,Encampment,Encampment Ongoing,NaT,Mass arrests,SJP UMN,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Twin Cities, Minnesotta"
5,Brown University,Providence,Rhode Island,U.S.,41.826800,-71.402500,https://drive.google.com/drive/folders/1qXcHVI...,Encampment,Encampment Ongoing,2024-04-24,,Brown Palestine Solidarity Caucus,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Providence, Rhode Island"
6,Cal Poly Humboldt,Arcata,California,U.S.,40.874700,-124.078900,https://drive.google.com/drive/folders/167OwEb...,Encampment,Encampment Ongoing,2024-04-23,Police raid,@LetitiaYund,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Arcata, California"
7,"University of North Carolina, Chapel Hill",Chapell Hill,North Carolina,U.S.,35.904900,-79.046900,https://drive.google.com/drive/folders/1xkfL7f...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",2024-04-19,,UNC SJP,NaT,NA,,,https://x.com/MiddleEastEye/status/17826392806...,"Chapell Hill, North Carolina"
8,University of Michigan,Ann Arbor,Michigan,U.S.,42.277100,-83.738200,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,2024-04-24,,SAFEumich,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Ann Arbor, Michigan"
9,Swarthmore College,Swarthmore,Pennsylvania,U.S.,39.903800,-75.352700,https://drive.google.com/drive/folders/19-TH5R...,Encampment,Encampment Ongoing,2024-04-23,,,NaT,NA,,,https://www.swarthmore.edu/presidents-office/p...,"Swarthmore, Pennsylvania"


In [61]:
directory_path = '/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/'
directory_files = os.listdir(directory_path)

list_of_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

file_list = []
for i in encampment_data['University Name']:
  list_of_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/' + str(i) + '/*') # * means all if need specific format then *.csv
  if list_of_files:
        latest_file = max(list_of_files, key=os.path.getctime)
  else:
      latest_file = None  # or use a default value for latest_file
  names = {}
  names['University Name'] = i
  names['Image Path'] = latest_file
  file_list.append(names)

file_df = pd.DataFrame(file_list)

encampment_data2 = pd.merge(encampment_data, file_df, on = 'University Name')
encampment_data2 = encampment_data2.dropna(subset=['Latitude', 'Longitude'])
encampment_data2
# for imageName in encampment_data['University Name']:
#     frames = []
#     imageName = imageName.split(',')
#     for i in imageName:
#         frames.append(directory_path + str(i) + '/' + '*.jpeg')
#     with imageio.get_writer(directory_path + str(i) + '/' + '*.gif', mode='I', duration=2) as writer:
#         for filename in frames:
#             image = cv2.resize(imageio.imread(filename), (width, height))
#             writer.append_data(image)
#             writer.append_data(image)

/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/American University of Egypt


,University Name,City,State,Country,Latitude,Longitude,Photo Location,Category,Status,Encampment Start Date,Police/State Violence,Image Source,Date of raid/arrests,Number of Arrests,Police Violence,,Source,Location,Image Path
0,Columbia University & Barnard College,New York,New York,U.S.,40.807384,-73.963036,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,2024-04-17,Mass arrests,PYM NYC,2024-04-18,114+,,,https://www.instagram.com/reel/C53zTS0uLxg/?ut...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
1,The New School,New York,New York,U.S.,40.734677,-73.991509,https://drive.google.com/drive/folders/12-V7pk...,Encampment,Encampment Ongoing,2024-04-21,,TNS SJP,NaT,NA,,,https://www.instagram.com/reel/C6JjZykLzGB/?ut...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
2,New York University (NYU),New York,New York,U.S.,40.729675,-73.996925,https://drive.google.com/drive/folders/12-V7pk...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",2024-04-22,Mass arrests,National SJP,2024-04-22,115+,@AshAgony,,https://www.instagram.com/p/C6DrwzRuxhV/?utm_s...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
3,Yale University,New Haven,Connecticut,U.S.,41.316307,-72.922585,https://drive.google.com/drive/folders/1x071zz...,Encampment,Encampment Dispersed,2024-04-20,Mass arrests,Yalies4palestine,2024-04-22,47+,,,https://www.instagram.com/p/C6EF_mnL9k0/,"New Haven, Connecticut",/content/drive/MyDrive/Colab Notebooks/Gaza So...
4,University of Minnesotta,Twin Cities,Minnesotta,U.S.,44.974000,-93.227700,https://drive.google.com/drive/folders/1U_tKJC...,Encampment,Encampment Ongoing,NaT,Mass arrests,SJP UMN,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Twin Cities, Minnesotta",/content/drive/MyDrive/Colab Notebooks/Gaza So...
5,Brown University,Providence,Rhode Island,U.S.,41.826800,-71.402500,https://drive.google.com/drive/folders/1qXcHVI...,Encampment,Encampment Ongoing,2024-04-24,,Brown Palestine Solidarity Caucus,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Providence, Rhode Island",/content/drive/MyDrive/Colab Notebooks/Gaza So...
6,Cal Poly Humboldt,Arcata,California,U.S.,40.874700,-124.078900,https://drive.google.com/drive/folders/167OwEb...,Encampment,Encampment Ongoing,2024-04-23,Police raid,@LetitiaYund,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Arcata, California",/content/drive/MyDrive/Colab Notebooks/Gaza So...
7,"University of North Carolina, Chapel Hill",Chapell Hill,North Carolina,U.S.,35.904900,-79.046900,https://drive.google.com/drive/folders/1xkfL7f...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",2024-04-19,,UNC SJP,NaT,NA,,,https://x.com/MiddleEastEye/status/17826392806...,"Chapell Hill, North Carolina",/content/drive/MyDrive/Colab Notebooks/Gaza So...
8,University of Michigan,Ann Arbor,Michigan,U.S.,42.277100,-83.738200,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,2024-04-24,,SAFEumich,NaT,NA,,,https://www.motherjones.com/politics/2024/04/g...,"Ann Arbor, Michigan",/content/drive/MyDrive/Colab Notebooks/Gaza So...
9,Swarthmore College,Swarthmore,Pennsylvania,U.S.,39.903800,-75.352700,https://drive.google.com/drive/folders/19-TH5R...,Encampment,Encampment Ongoing,2024-04-23,,,NaT,NA,,,https://www.swarthmore.edu/presidents-office/p...,"Swarthmore, Pennsylvania",/content/drive/MyDrive/Colab Notebooks/Gaza So...


In [63]:
usa_coord = [37.0902, -95.7129] # Latitude, Longitude coord in decimal degrees

# Create USA map with folium wrapper around leaflet.js
usa_map = folium.Map(location=usa_coord, zoom_start=4.4)
folium.TileLayer('cartodbpositron').add_to(usa_map)

# Encampment Markers
## Map Icon
pal_flag = "/content/drive/My Drive/673813_map_map marker_national_country_palestinian_pin_flag_palestine.png"
icon = folium.CustomIcon(pal_flag, icon_size=(60, 60))  # Adjust icon size as needed , icon_size=(50, 50)

## Map Loop
for index, row in encampment_data2.iterrows():

  if row['Image Path'] is not None:
    with open(row['Image Path'], "rb") as img_file:
      encoded_image = base64.b64encode(img_file.read()).decode('utf-8')
  else:
    print(f"Warning: Image Path is None for row {index}")

  popup_html = f"""
        <!DOCTYPE html>
        <html>
        <h3 align="left" style="font-family:Calibri; color:red"><strong><u>{row['University Name']}</u><strong>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
        </h3>
        <h4 align="left" style="font-family:Calibri; color:green"> <strong>Location:</strong> {row['Location']}
        </h4>
        <h4 align="left" style="font-family:Calibri; color:green"> <strong>Encampment Start Date:</strong> {row['Encampment Start Date']}
        </h4>
        <h4 align="left" style="font-family:Calibri; color:green"> <strong>Status:</strong> {row['Status']}
        </h4>
        <h4 align="left" style="font-family:Calibri; color:green"> <strong>Date of Raid/Arrests:</strong> {row['Date of raid/arrests']}
        </h4>
        <h4 align="left" style="font-family:Calibri; color:green"> <strong>Number of Arrests:</strong> {row['Number of Arrests']}
        </h4>
        <img src="data:image/jpeg;base64,{encoded_image}" alt="Image" style="width:250px;height:200px;">

        """

  icon = folium.CustomIcon(pal_flag, icon_size=(60, 60))  # Adjust icon size as needed , icon_size=(50, 50)
  popup = folium.Popup(popup_html, max_width=300)
  folium.vector_layers.Marker(location=[row['Latitude'],row['Longitude']],
                                popup = popup,
                                icon = icon,
                                tooltip=str(row['University Name'])).add_to(usa_map)

usa_map

In [52]:
usa_map.save("/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/encampments_map.html")

In [ ]:
usa_map